#0.初期設定

In [ ]:
#*******今回の実行の設定*******

#プロジェクトのフォルダ　※BASE_DIRを、自らの環境に合わせて修正してください
BASE_DIR = '/content/drive/MyDrive/project01'


In [22]:
#*******Googleドライブへの接続*******
from google.colab import drive
drive.mount('/content/drive')

#*******ライブラリのインポート*******
import sys
import os
import pandas as pd
import numpy as np
import glob
from scipy.io import loadmat

#*******定数設定*******
TRAIN_MAT_DIR1 = os.path.join(BASE_DIR, 'data1/train')
TRAIN_SAVE_DIR1 = os.path.join(BASE_DIR, 'data1/modeling_data')
TRAIN_MAT_DIR2 = os.path.join(BASE_DIR, 'data2/train')
TRAIN_SAVE_DIR2 = os.path.join(BASE_DIR, 'data2/modeling_data')

TEST_MAT_DIR1 = os.path.join(BASE_DIR, 'data1/test')
TEST_SAVE_DIR1 = os.path.join(BASE_DIR, 'data1/modeling_data_val')
TEST_MAT_DIR2 = os.path.join(BASE_DIR, 'data2/test')
TEST_SAVE_DIR2 = os.path.join(BASE_DIR, 'data2/modeling_data_val')

#被験者
SUBJECTS = ['subject0','subject1','subject2','subject3','subject4']
#試行
TRAIN_MATS = ['train1.mat','train2.mat','train3.mat']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1.学習用データ作成

In [ ]:
#*******学習用データ作成関数*******
def make_train_data(mat_dir, save_dir):

    for subject_id in SUBJECTS:
        for mat in TRAIN_MATS:
            os.makedirs(os.path.join(save_dir, subject_id, mat), exist_ok=True)
            labels = {
                '11': 'frontside_kickturn',
                '12': 'backside_kickturn',
                '13': 'pumping',
                '21': 'frontside_kickturn',
                '22': 'backside_kickturn',
                '23': 'pumping'
            }
            counts = {'frontside_kickturn': 0, 'backside_kickturn': 0, 'pumping': 0}
            for fname in [mat]:
                data = loadmat(os.path.join(mat_dir, subject_id, fname))
                event_data = data['event'][0, 0]  # ネストを解除
                event_types = event_data[0]
                event_times = event_data[1]
                event_indices = event_data[2]

                event_df = pd.DataFrame({
                    'type': event_types.flatten(),
                    'init_time': event_times.flatten(),
                    'init_index': event_indices.flatten()
                })

                # 'times'をフラットに変換
                if data['times'].ndim == 3:
                    data['times'] = data['times'].squeeze()

                # 'times' を2次元配列に変換
                times = data['times'].squeeze()  # 余分な次元を削除
                times = times.reshape(1, -1)  # 2次元配列に変形

                ts = pd.DataFrame(np.concatenate([times, data['data']], axis=0).T, columns=['Time'] + list(data['ch_labels'].flatten()))
                for i, d in event_df.iterrows():
                    it = d['init_time'] + 0.2
                    et = d['init_time'] + 0.7
                    event_type = str(int(d['type']))
                    ts_seg = ts[(ts['Time'] >= it * 1e3) & (ts['Time'] <= et * 1e3)]
                    if len(ts_seg)>250: ts_seg = ts_seg[0:250]

                    if not os.path.exists(os.path.join(save_dir, subject_id, labels[event_type])):
                        os.makedirs(os.path.join(save_dir, subject_id, mat, labels[event_type]), exist_ok=True)
                    del ts_seg['Time']
                    ts_seg.to_csv(os.path.join(save_dir, subject_id, mat, labels[event_type], '{:03d}.csv'.format(counts[labels[event_type]])), index=False, header=False)

                    counts[labels[event_type]] += 1


#*******データ生成の実行*******
make_train_data(TRAIN_MAT_DIR1, TRAIN_SAVE_DIR1)
make_train_data(TRAIN_MAT_DIR2, TRAIN_SAVE_DIR2)

#2.推論用データ作成

In [24]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os

#*******評価用データ作成関数*******
def make_val_data(mat_dir, save_dir):

    for subject in SUBJECTS:
        # MATファイルの読み込み
        mat = loadmat(os.path.join(mat_dir, f"{subject}.mat"))
        data = mat["data"]
        batch = data.shape[0]

        # 保存先フォルダ作成
        os.makedirs(os.path.join(save_dir, subject, f"{subject}.mat", "unknown"), exist_ok=True)

        for n in range(batch):
            data_72_250 = data[n]

            # DataFrameとして変換
            df = pd.DataFrame(data_72_250.T)  #転置して72列×250行に

            # CSVとして保存
            fname = f"000{n}"[-3:]+".csv"
            save_path = os.path.join(save_dir, subject, f"{subject}.mat", "unknown", fname)
            df.to_csv(save_path, index=False, header=False)  #インデックスとヘッダーを除く


#*******データ生成の実行*******
make_val_data(TEST_MAT_DIR1, TEST_SAVE_DIR1)
make_val_data(TEST_MAT_DIR2, TEST_SAVE_DIR2)